In [1]:
from pydantic import BaseModel, Field
from typing import List, Dict, Tuple
from uuid import UUID, uuid4
import json
import random

class City(BaseModel):
    name: UUID = Field(default_factory=uuid4)
    coordinates: List[float]
    interests: List[UUID]
    country: str

class Route(BaseModel):
    city_pair: Tuple[UUID, UUID]  # 使用元组确保顺序一致性
    steps: List[UUID]
    distance: float

def generate_random_lat_lon():
    lat = round(random.uniform(-90, 90), 2)
    lon = round(random.uniform(-180, 180), 2)
    return [lat, lon]

def initial_data():
    """初始化数据并保存到JSON文件"""
    # 生成城市数据
    cities_dict = {"aaa": {}, "bbb": {}}
    all_cities = []
    
    # 为每个国家生成10个城市
    for country in ["aaa", "bbb"]:
        for _ in range(10):
            city = City(
                name=uuid4(),
                coordinates=generate_random_lat_lon(),
                interests=[uuid4() for _ in range(random.randint(1,10))],
                country=country
            )
            cities_dict[country][str(city.name)] = json.loads(city.model_dump_json())  # 直接存储为JSON对象而非字符串
            all_cities.append(city)
    
    # 生成路线数据
    routes_dict = {}
    for i in range(len(all_cities)):
        for j in range(i+1, len(all_cities)):
            city1, city2 = all_cities[i], all_cities[j]
            same_country = city1.country == city2.country
            
            steps_count = random.randint(1, 3) if same_country else random.randint(2, 5)
            distance = random.uniform(0, 100) if same_country else random.uniform(100, 300)
            
            # 确保路线键的一致性 - 使用排序后的UUID
            city_ids = sorted([str(city1.name), str(city2.name)])
            route_key = f"{city_ids[0]}_{city_ids[1]}"
            
            route = Route(
                city_pair=(city1.name, city2.name),  # 使用元组
                steps=[uuid4() for _ in range(steps_count)],
                distance=round(distance, 2)
            )
            
            # 只存储一次路线，避免重复
            routes_dict[route_key] = json.loads(route.model_dump_json())  # 直接存储为JSON对象而非字符串
    
    # 保存数据到JSON文件
    with open('cities.json', 'w') as f:
        json.dump(cities_dict, f, indent=2)
    
    with open('routes.json', 'w') as f:
        json.dump(routes_dict, f, indent=2)

def load_data():
    """从JSON文件加载数据并将其转换为Pydantic对象"""
    try:
        with open('cities.json', 'r') as f:
            cities_dict = json.load(f)
        with open('routes.json', 'r') as f:
            routes_dict = json.load(f)

        # 将JSON对象转换为City和Route对象
        cities = {}
        for country, city_items in cities_dict.items():
            cities[country] = {city_id: City(**city_data) for city_id, city_data in city_items.items()}

        routes = {}
        for route_key, route_data in routes_dict.items():
            city1_id, city2_id = route_key.split("_")
            # 更新路由键格式，确保查询时可以找到对应路线
            routes[route_key] = Route(**route_data)
            # 为便于双向查询，添加反向查询支持
            reverse_key = f"{city2_id}_{city1_id}"
            if reverse_key not in routes:
                routes[reverse_key] = routes[route_key]

        return cities, routes
    except FileNotFoundError:
        initial_data()
        return load_data()

# 用于查找两个城市之间的路线
def find_route(routes, city1_id, city2_id):
    """查找两个城市之间的路线"""
    # 尝试两种可能的键格式
    key1 = f"{city1_id}_{city2_id}"
    key2 = f"{city2_id}_{city1_id}"
    
    if key1 in routes:
        return routes[key1]
    elif key2 in routes:
        return routes[key2]
    else:
        return None

# === aaa国的API ===

def get_distance_between_cities_aaa(city1_id: UUID, city2_id: UUID):
    cities, routes = load_data()
    
    # 检查城市是否在aaa国
    if str(city1_id) not in cities["aaa"]:
        return "错误：第一个城市不在aaa国"
    if str(city2_id) not in cities["aaa"]:
        return "错误：第二个城市不在aaa国"
    
    route = find_route(routes, str(city1_id), str(city2_id))
    return route.distance if route else "错误：未找到这两个城市之间的路线"

def get_route_between_cities_aaa(city1_id: UUID, city2_id: UUID):
    cities, routes = load_data()
    
    # 检查城市是否在aaa国
    if str(city1_id) not in cities["aaa"]:
        return "错误：第一个城市不在aaa国"
    if str(city2_id) not in cities["aaa"]:
        return "错误：第二个城市不在aaa国"
    
    route = find_route(routes, str(city1_id), str(city2_id))
    return route if route else "错误：未找到这两个城市之间的路线"

def get_city_coordinates_aaa(city_id: UUID):
    cities, _ = load_data()
    city_str_id = str(city_id)
    if city_str_id in cities["aaa"]:
        return cities["aaa"][city_str_id].coordinates
    return "错误：在aaa国找不到该城市"

def get_nearby_locations_aaa(city_id: UUID, max_distance: float = 50):
    cities, routes = load_data()
    city_str_id = str(city_id)
    
    if city_str_id not in cities["aaa"]:
        return "错误：在aaa国找不到该城市"
    
    nearby_cities = []
    for route_key, route in routes.items():
        city_ids = route_key.split('_')
        if city_str_id in city_ids and route.distance <= max_distance:
            other_id = city_ids[1] if city_ids[0] == city_str_id else city_ids[0]
            if other_id in cities["aaa"]:
                nearby_cities.append(cities["aaa"][other_id])
    
    return nearby_cities if nearby_cities else "未找到附近地点"

# === bbb国的API ===

def get_distance_between_cities_bbb(city1_id: UUID, city2_id: UUID):
    cities, routes = load_data()
    
    if str(city1_id) not in cities["bbb"]:
        return "错误：第一个城市不在bbb国"
    if str(city2_id) not in cities["bbb"]:
        return "错误：第二个城市不在bbb国"
    
    route = find_route(routes, str(city1_id), str(city2_id))
    return route.distance if route else "错误：未找到这两个城市之间的路线"

def get_route_between_cities_bbb(city1_id: UUID, city2_id: UUID):
    cities, routes = load_data()
    
    # 使用字符串表示的UUID进行字典查找
    city1_str = str(city1_id)
    city2_str = str(city2_id)
    
    # 检查城市是否在bbb国
    if city1_str not in cities["bbb"]:
        return "错误：第一个城市不在bbb国"
    if city2_str not in cities["bbb"]:
        return "错误：第二个城市不在bbb国"
    
    route = find_route(routes, city1_str, city2_str)
    return route if route else "错误：未找到这两个城市之间的路线"

def get_city_coordinates_bbb(city_id: UUID):
    cities, _ = load_data()
    city_str_id = str(city_id)
    if city_str_id in cities["bbb"]:
        return cities["bbb"][city_str_id].coordinates
    return "错误：在bbb国找不到该城市"

def get_nearby_locations_bbb(city_id: UUID, max_distance: float = 50):
    cities, routes = load_data()
    city_str_id = str(city_id)
    
    if city_str_id not in cities["bbb"]:
        return "错误：在bbb国找不到该城市"
    
    nearby_cities = []
    for route_key, route in routes.items():
        city_ids = route_key.split('_')
        if city_str_id in city_ids and route.distance <= max_distance:
            other_id = city_ids[1] if city_ids[0] == city_str_id else city_ids[0]
            if other_id in cities["bbb"]:
                nearby_cities.append(cities["bbb"][other_id])
    
    return nearby_cities if nearby_cities else "未找到附近地点"

# === 跨国API ===

def get_cross_country_distance(city1_id: UUID, city2_id: UUID):
    cities, routes = load_data()
    city1_str = str(city1_id)
    city2_str = str(city2_id)
    
    # 查找城市所属国家
    city1_country = None
    city2_country = None
    
    for country in cities:
        if city1_str in cities[country]:
            city1_country = country
        if city2_str in cities[country]:
            city2_country = country
    
    # 验证条件
    if not city1_country:
        return "错误：找不到第一个城市"
    if not city2_country:
        return "错误：找不到第二个城市"
    if city1_country == city2_country:
        return f"错误：两个城市都在{city1_country}国，请使用本国API"
    
    route = find_route(routes, city1_str, city2_str)
    return route.distance if route else "错误：未找到这两个城市之间的路线"

def get_city_country(city_id: UUID):
    cities, _ = load_data()
    city_str_id = str(city_id)
    
    for country, country_cities in cities.items():
        if city_str_id in country_cities:
            return country
    
    return "错误：未找到该城市"
# 示例使用
if __name__ == "__main__":
    initial_data()
    cities, routes = load_data()
    print(f"加载了 {sum(len(country_cities) for country_cities in cities.values())} 个城市")
    print(f"加载了 {len(routes)} 条路线")
    


加载了 20 个城市
加载了 380 条路线
城市 796abc07-88d0-45fb-9bab-e7bfb3aa9ee1 和 50b485db-22de-4bb8-a01a-71d80c3169a1 之间的路线距离: 42.09


In [1]:
from pydantic import BaseModel, Field
from typing import List, Dict, Tuple
from uuid import UUID, uuid4
import json
import random

class City(BaseModel):
    name: UUID = Field(default_factory=uuid4)
    coordinates: List[float]
    interests: List[UUID]
    country: str

class Route(BaseModel):
    city_pair: Tuple[UUID, UUID]  # 使用元组确保顺序一致性
    steps: List[UUID]
    distance: float

def generate_random_lat_lon():
    """生成随机的经纬度坐标
    
    Returns:
        List[float]: 包含纬度(-90到90)和经度(-180到180)的列表
    """
    lat = round(random.uniform(-90, 90), 2)
    lon = round(random.uniform(-180, 180), 2)
    return [lat, lon]

def initial_data():
    """初始化数据并保存到JSON文件"""
    # 生成城市数据
    cities_dict = {"aaa": {}, "bbb": {}}
    all_cities = []
    
    # 为每个国家生成10个城市
    for country in ["aaa", "bbb"]:
        for _ in range(10):
            city = City(
                name=uuid4(),
                coordinates=generate_random_lat_lon(),
                interests=[uuid4() for _ in range(random.randint(1,10))],
                country=country
            )
            cities_dict[country][str(city.name)] = json.loads(city.model_dump_json())  # 直接存储为JSON对象而非字符串
            all_cities.append(city)
    
    # 生成路线数据
    routes_dict = {}
    for i in range(len(all_cities)):
        for j in range(i+1, len(all_cities)):
            city1, city2 = all_cities[i], all_cities[j]
            same_country = city1.country == city2.country
            
            steps_count = random.randint(1, 3) if same_country else random.randint(2, 5)
            distance = random.uniform(0, 100) if same_country else random.uniform(100, 300)
            
            # 确保路线键的一致性 - 使用排序后的UUID
            city_ids = sorted([str(city1.name), str(city2.name)])
            route_key = f"{city_ids[0]}_{city_ids[1]}"
            
            route = Route(
                city_pair=(city1.name, city2.name),  # 使用元组
                steps=[uuid4() for _ in range(steps_count)],
                distance=round(distance, 2)
            )
            
            # 只存储一次路线，避免重复
            routes_dict[route_key] = json.loads(route.model_dump_json())  # 直接存储为JSON对象而非字符串
    
    # 保存数据到JSON文件
    with open('cities.json', 'w') as f:
        json.dump(cities_dict, f, indent=2)
    
    with open('routes.json', 'w') as f:
        json.dump(routes_dict, f, indent=2)

def load_data():
    """从JSON文件加载数据并将其转换为Pydantic对象"""
    try:
        with open('cities.json', 'r') as f:
            cities_dict = json.load(f)
        with open('routes.json', 'r') as f:
            routes_dict = json.load(f)

        # 将JSON对象转换为City和Route对象
        cities = {}
        for country, city_items in cities_dict.items():
            cities[country] = {city_id: City(**city_data) for city_id, city_data in city_items.items()}

        routes = {}
        for route_key, route_data in routes_dict.items():
            city1_id, city2_id = route_key.split("_")
            # 更新路由键格式，确保查询时可以找到对应路线
            routes[route_key] = Route(**route_data)
            # 为便于双向查询，添加反向查询支持
            reverse_key = f"{city2_id}_{city1_id}"
            if reverse_key not in routes:
                routes[reverse_key] = routes[route_key]

        return cities, routes
    except FileNotFoundError:
        initial_data()
        return load_data()

# 用于查找两个城市之间的路线
def find_route(routes, city1_id, city2_id):
    """查找两个城市之间的路线"""
    # 尝试两种可能的键格式
    key1 = f"{city1_id}_{city2_id}"
    key2 = f"{city2_id}_{city1_id}"
    
    if key1 in routes:
        return routes[key1]
    elif key2 in routes:
        return routes[key2]
    else:
        return None

# === aaa国的API ===

def get_distance_between_cities_aaa(city1_id: UUID, city2_id: UUID):
    """获取aaa国家中两个城市之间的距离
    
    Args:
        city1_id (UUID): 第一个城市的ID
        city2_id (UUID): 第二个城市的ID
        
    Returns:
        float或str: 如果两个城市都在aaa国且路线存在，返回距离；否则返回错误信息
    """
    cities, routes = load_data()
    
    # 检查城市是否在aaa国
    if str(city1_id) not in cities["aaa"]:
        return "错误：第一个城市不在aaa国"
    if str(city2_id) not in cities["aaa"]:
        return "错误：第二个城市不在aaa国"
    
    route = find_route(routes, str(city1_id), str(city2_id))
    return route.distance if route else "错误：未找到这两个城市之间的路线"

def get_route_between_cities_aaa(city1_id: UUID, city2_id: UUID):
    """获取aaa国家中两个城市之间的路线信息
    
    Args:
        city1_id (UUID): 第一个城市的ID
        city2_id (UUID): 第二个城市的ID
        
    Returns:
        Route或str: 如果两个城市都在aaa国且路线存在，返回路线对象；否则返回错误信息
    """
    cities, routes = load_data()
    
    # 检查城市是否在aaa国
    if str(city1_id) not in cities["aaa"]:
        return "错误：第一个城市不在aaa国"
    if str(city2_id) not in cities["aaa"]:
        return "错误：第二个城市不在aaa国"
    
    route = find_route(routes, str(city1_id), str(city2_id))
    return route if route else "错误：未找到这两个城市之间的路线"

def get_city_coordinates_aaa(city_id: UUID):
    """获取aaa国家中指定城市的坐标
    
    Args:
        city_id (UUID): 城市的ID
        
    Returns:
        List[float]或str: 如果城市在aaa国，返回坐标；否则返回错误信息
    """
    cities, _ = load_data()
    city_str_id = str(city_id)
    if city_str_id in cities["aaa"]:
        return cities["aaa"][city_str_id].coordinates
    return "错误：在aaa国找不到该城市"

def get_nearby_locations_aaa(city_id: UUID, max_distance: float = 50):
    """获取aaa国家中指定城市附近的所有地点
    
    Args:
        city_id (UUID): 目标城市的ID
        max_distance (float, optional): 最大距离阈值，默认为50
        
    Returns:
        List[City]或str: 如果找到符合条件的城市，返回城市列表；否则返回错误信息
    """
    cities, routes = load_data()
    city_str_id = str(city_id)
    
    if city_str_id not in cities["aaa"]:
        return "错误：在aaa国找不到该城市"
    
    nearby_cities = []
    for route_key, route in routes.items():
        city_ids = route_key.split('_')
        if city_str_id in city_ids and route.distance <= max_distance:
            other_id = city_ids[1] if city_ids[0] == city_str_id else city_ids[0]
            if other_id in cities["aaa"]:
                nearby_cities.append(cities["aaa"][other_id])
    
    return nearby_cities if nearby_cities else "未找到附近地点"

# === bbb国的API ===

def get_distance_between_cities_bbb(city1_id: UUID, city2_id: UUID):
    """获取bbb国家中两个城市之间的距离
    
    Args:
        city1_id (UUID): 第一个城市的ID
        city2_id (UUID): 第二个城市的ID
        
    Returns:
        float或str: 如果两个城市都在bbb国且路线存在，返回距离；否则返回错误信息
    """
    cities, routes = load_data()
    
    if str(city1_id) not in cities["bbb"]:
        return "错误：第一个城市不在bbb国"
    if str(city2_id) not in cities["bbb"]:
        return "错误：第二个城市不在bbb国"
    
    route = find_route(routes, str(city1_id), str(city2_id))
    return route.distance if route else "错误：未找到这两个城市之间的路线"

def get_route_between_cities_bbb(city1_id: UUID, city2_id: UUID):
    """获取bbb国家中两个城市之间的路线信息
    
    Args:
        city1_id (UUID): 第一个城市的ID
        city2_id (UUID): 第二个城市的ID
        
    Returns:
        Route或str: 如果两个城市都在bbb国且路线存在，返回路线对象；否则返回错误信息
    """
    cities, routes = load_data()
    
    # 使用字符串表示的UUID进行字典查找
    city1_str = str(city1_id)
    city2_str = str(city2_id)
    
    # 检查城市是否在bbb国
    if city1_str not in cities["bbb"]:
        return "错误：第一个城市不在bbb国"
    if city2_str not in cities["bbb"]:
        return "错误：第二个城市不在bbb国"
    
    route = find_route(routes, city1_str, city2_str)
    return route if route else "错误：未找到这两个城市之间的路线"

def get_city_coordinates_bbb(city_id: UUID):
    """获取bbb国家中指定城市的坐标
    
    Args:
        city_id (UUID): 城市的ID
        
    Returns:
        List[float]或str: 如果城市在bbb国，返回坐标；否则返回错误信息
    """
    cities, _ = load_data()
    city_str_id = str(city_id)
    if city_str_id in cities["bbb"]:
        return cities["bbb"][city_str_id].coordinates
    return "错误：在bbb国找不到该城市"

def get_nearby_locations_bbb(city_id: UUID, max_distance: float = 50):
    """获取bbb国家中指定城市附近的所有地点
    
    Args:
        city_id (UUID): 目标城市的ID
        max_distance (float, optional): 最大距离阈值，默认为50
        
    Returns:
        List[City]或str: 如果找到符合条件的城市，返回城市列表；否则返回错误信息
    """
    cities, routes = load_data()
    city_str_id = str(city_id)
    
    if city_str_id not in cities["bbb"]:
        return "错误：在bbb国找不到该城市"
    
    nearby_cities = []
    for route_key, route in routes.items():
        city_ids = route_key.split('_')
        if city_str_id in city_ids and route.distance <= max_distance:
            other_id = city_ids[1] if city_ids[0] == city_str_id else city_ids[0]
            if other_id in cities["bbb"]:
                nearby_cities.append(cities["bbb"][other_id])
    
    return nearby_cities if nearby_cities else "未找到附近地点"

# === 跨国API ===

def get_cross_country_distance(city1_id: UUID, city2_id: UUID):
    """获取不同国家的两个城市之间的距离
    
    Args:
        city1_id (UUID): 第一个城市的ID
        city2_id (UUID): 第二个城市的ID
        
    Returns:
        float或str: 如果两个城市在不同国家且路线存在，返回距离；否则返回错误信息
    """
    cities, routes = load_data()
    city1_str = str(city1_id)
    city2_str = str(city2_id)
    
    # 查找城市所属国家
    city1_country = None
    city2_country = None
    
    for country in cities:
        if city1_str in cities[country]:
            city1_country = country
        if city2_str in cities[country]:
            city2_country = country
    
    # 验证条件
    if not city1_country:
        return "错误：找不到第一个城市"
    if not city2_country:
        return "错误：找不到第二个城市"
    if city1_country == city2_country:
        return f"错误：两个城市都在{city1_country}国，请使用本国API"
    
    route = find_route(routes, city1_str, city2_str)
    return route.distance if route else "错误：未找到这两个城市之间的路线"

def get_city_country(city_id: UUID):
    """获取指定城市所属的国家
    
    Args:
        city_id (UUID): 城市的ID
        
    Returns:
        str: 城市所属的国家，或者错误信息
    """
    cities, _ = load_data()
    city_str_id = str(city_id)
    
    for country, country_cities in cities.items():
        if city_str_id in country_cities:
            return country
    
    return "错误：未找到该城市"

# 示例使用
if __name__ == "__main__":
    initial_data()
    cities, routes = load_data()
    print(f"加载了 {sum(len(country_cities) for country_cities in cities.values())} 个城市")
    print(f"加载了 {len(routes)} 条路线")

加载了 20 个城市
加载了 380 条路线


In [2]:
import random
from uuid import UUID
import json

# 假设以上所有代码都已导入（主代码库+API函数）

def test_all_apis():
    print("初始化数据...")
    # 确保数据已经生成
    cities, routes = load_data()
    
    # === 获取测试数据 ===
    print("\n准备测试数据...")
    
    # 获取aaa国的两个城市
    aaa_city_ids = list(cities["aaa"].keys())
    if len(aaa_city_ids) >= 2:
        aaa_city1_id = aaa_city_ids[0]
        aaa_city2_id = aaa_city_ids[1]
        print(f"已选择aaa国城市: {aaa_city1_id[:8]}... 和 {aaa_city2_id[:8]}...")
    else:
        print("错误: aaa国城市数量不足")
        return
    
    # 获取bbb国的两个城市
    bbb_city_ids = list(cities["bbb"].keys())
    if len(bbb_city_ids) >= 2:
        bbb_city1_id = bbb_city_ids[0]
        bbb_city2_id = bbb_city_ids[1]
        print(f"已选择bbb国城市: {bbb_city1_id[:8]}... 和 {bbb_city2_id[:8]}...")
    else:
        print("错误: bbb国城市数量不足")
        return
    
    # === 测试aaa国API ===
    print("\n=== 测试aaa国API ===")
    
    # 测试获取城市坐标
    print("\n1. 测试 get_city_coordinates_aaa:")
    # 正确案例
    coords = get_city_coordinates_aaa(UUID(aaa_city1_id))
    print(f"城市坐标: {coords}")
    # 错误案例 - 使用bbb国城市
    wrong_coords = get_city_coordinates_aaa(UUID(bbb_city1_id))
    print(f"使用bbb国城市ID的结果: {wrong_coords}")
    
    # 测试城市间距离
    print("\n2. 测试 get_distance_between_cities_aaa:")
    # 正确案例
    distance = get_distance_between_cities_aaa(UUID(aaa_city1_id), UUID(aaa_city2_id))
    print(f"城市间距离: {distance}")
    # 错误案例 - 混合国家
    mixed_distance = get_distance_between_cities_aaa(UUID(aaa_city1_id), UUID(bbb_city1_id))
    print(f"混合国家的结果: {mixed_distance}")
    
    # 测试城市间路线
    print("\n3. 测试 get_route_between_cities_aaa:")
    # 正确案例
    route = get_route_between_cities_aaa(UUID(aaa_city1_id), UUID(aaa_city2_id))
    print(f"城市间路线: {route}")
    
    # 测试附近位置
    print("\n4. 测试 get_nearby_locations_aaa:")
    # 使用大距离以确保能找到附近城市
    nearby = get_nearby_locations_aaa(UUID(aaa_city1_id), 500)
    if isinstance(nearby, list):
        print(f"找到了 {len(nearby)} 个附近城市")
        if nearby:
            print(f"第一个附近城市: {nearby[0].name}")
    else:
        print(f"附近位置结果: {nearby}")
    
    # === 测试bbb国API ===
    print("\n=== 测试bbb国API ===")
    
    # 测试获取城市坐标
    print("\n1. 测试 get_city_coordinates_bbb:")
    coords = get_city_coordinates_bbb(UUID(bbb_city1_id))
    print(f"城市坐标: {coords}")
    
    # 测试城市间距离
    print("\n2. 测试 get_distance_between_cities_bbb:")
    distance = get_distance_between_cities_bbb(UUID(bbb_city1_id), UUID(bbb_city2_id))
    print(f"城市间距离: {distance}")
    
    # 测试城市间路线
    print("\n3. 测试 get_route_between_cities_bbb:")
    route = get_route_between_cities_bbb(UUID(bbb_city1_id), UUID(bbb_city2_id))
    if not isinstance(route, str):
        print(f"城市间路线: 距离 {route.distance}，步骤数 {len(route.steps)}")
    else:
        print(f"城市间路线: {route}")
    
    # 测试附近位置
    print("\n4. 测试 get_nearby_locations_bbb:")
    nearby = get_nearby_locations_bbb(UUID(bbb_city1_id), 500)
    if isinstance(nearby, list):
        print(f"找到了 {len(nearby)} 个附近城市")
    else:
        print(f"附近位置结果: {nearby}")
    
    # === 测试跨国API ===
    print("\n=== 测试跨国API ===")
    
    # 测试城市所属国家
    print("\n1. 测试 get_city_country:")
    aaa_country = get_city_country(UUID(aaa_city1_id))
    print(f"aaa城市所属国家: {aaa_country}")
    bbb_country = get_city_country(UUID(bbb_city1_id))
    print(f"bbb城市所属国家: {bbb_country}")
    
    # 测试跨国距离
    print("\n2. 测试 get_cross_country_distance:")
    # 正确案例 - 跨国
    cross_distance = get_cross_country_distance(UUID(aaa_city1_id), UUID(bbb_city1_id))
    print(f"跨国距离: {cross_distance}")
    # 错误案例 - 同国
    same_country = get_cross_country_distance(UUID(aaa_city1_id), UUID(aaa_city2_id))
    print(f"同国距离: {same_country}")

# 通过实际API测试查询不同城市的多种信息
def test_city_combination():
    print("\n=== 测试不同城市组合 ===")
    
    cities, routes = load_data()
    
    # 随机选择几个城市进行测试
    all_city_ids = []
    for country, country_cities in cities.items():
        for city_id in country_cities:
            all_city_ids.append((country, city_id))
    
    # 随机选择4个城市
    sample_size = min(4, len(all_city_ids))
    sample_cities = random.sample(all_city_ids, sample_size)
    
    print(f"随机选择了 {sample_size} 个城市进行测试")
    
    # 测试各种组合
    for i, (country1, city1_id) in enumerate(sample_cities):
        print(f"\n城市 {i+1}: {city1_id[:8]}... ({country1}国)")
        
        # 获取坐标
        if country1 == "aaa":
            coords = get_city_coordinates_aaa(UUID(city1_id))
        else:
            coords = get_city_coordinates_bbb(UUID(city1_id))
        print(f"坐标: {coords}")
        
        # 测试与其他城市的关系
        for j, (country2, city2_id) in enumerate(sample_cities):
            if i != j:
                print(f"  与城市 {j+1} ({city2_id[:8]}..., {country2}国) 的关系:")
                
                # 如果两个城市在同一国家
                if country1 == country2:
                    if country1 == "aaa":
                        distance = get_distance_between_cities_aaa(UUID(city1_id), UUID(city2_id))
                    else:
                        distance = get_distance_between_cities_bbb(UUID(city1_id), UUID(city2_id))
                    print(f"    同国距离: {distance}")
                # 如果两个城市在不同国家
                else:
                    distance = get_cross_country_distance(UUID(city1_id), UUID(city2_id))
                    print(f"    跨国距离: {distance}")

# 测试错误处理
def test_error_handling():
    print("\n=== 测试错误处理 ===")
    
    # 创建一个不存在的UUID
    fake_uuid = UUID('00000000-0000-0000-0000-000000000000')
    
    print("\n1. 测试不存在的城市:")
    print(f"aaa API结果: {get_city_coordinates_aaa(fake_uuid)}")
    print(f"bbb API结果: {get_city_coordinates_bbb(fake_uuid)}")
    print(f"跨国API结果: {get_city_country(fake_uuid)}")
    
    # 获取一个真实的aaa城市和bbb城市用于混合测试
    cities, _ = load_data()
    aaa_city_id = list(cities["aaa"].keys())[0]
    bbb_city_id = list(cities["bbb"].keys())[0]
    
    print("\n2. 测试错误的国家组合:")
    print(f"aaa API接收bbb城市: {get_distance_between_cities_aaa(UUID(bbb_city_id), UUID(bbb_city_id))}")
    print(f"bbb API接收aaa城市: {get_distance_between_cities_bbb(UUID(aaa_city_id), UUID(aaa_city_id))}")
    
    print("\n3. 测试同国跨国API:")
    print(f"跨国API接收同国城市: {get_cross_country_distance(UUID(aaa_city_id), UUID(list(cities['aaa'].keys())[1]))}")

if __name__ == "__main__":
    print("\n===== API 测试程序 =====\n")
    
    # 运行所有测试
    test_all_apis()
    test_city_combination()
    test_error_handling()
    
    print("\n===== 测试完成 =====")


===== API 测试程序 =====

初始化数据...

准备测试数据...
已选择aaa国城市: b79423ff... 和 1b05271b...
已选择bbb国城市: 4353070d... 和 82c71fba...

=== 测试aaa国API ===

1. 测试 get_city_coordinates_aaa:
城市坐标: [-50.53, -115.32]
使用bbb国城市ID的结果: 错误：在aaa国找不到该城市

2. 测试 get_distance_between_cities_aaa:
城市间距离: 66.17
混合国家的结果: 错误：第二个城市不在aaa国

3. 测试 get_route_between_cities_aaa:
城市间路线: city_pair=(UUID('b79423ff-c71d-4d93-8bf1-1e5807f10f7c'), UUID('1b05271b-660b-4e36-b574-e436d236dc7f')) steps=[UUID('3865b37e-89ab-4769-b8fa-006d87019e40'), UUID('49465f33-416d-4186-a237-0a3f25ef8275'), UUID('c1d37fc1-55c0-43ca-86e1-f760e6704433')] distance=66.17

4. 测试 get_nearby_locations_aaa:
找到了 18 个附近城市
第一个附近城市: 1b05271b-660b-4e36-b574-e436d236dc7f

=== 测试bbb国API ===

1. 测试 get_city_coordinates_bbb:
城市坐标: [14.33, 117.16]

2. 测试 get_distance_between_cities_bbb:
城市间距离: 65.93

3. 测试 get_route_between_cities_bbb:
城市间路线: 距离 65.93，步骤数 2

4. 测试 get_nearby_locations_bbb:
找到了 18 个附近城市

=== 测试跨国API ===

1. 测试 get_city_country:
aaa城市所属国家: aaa
bbb城市所属国家: b

In [28]:
print(cities)
print(routes)

{'aaa': {'6191718d-6319-4ade-85e2-2dbc842842c8': City(name=UUID('6191718d-6319-4ade-85e2-2dbc842842c8'), coordinates=[-32.19, 25.51], interests=[UUID('1669f1d1-c595-4b82-a9db-e59bcf222cca'), UUID('1183d50f-46af-45d4-8637-4025e7892502')], country='aaa'), 'abc911f5-bb28-44e6-97b9-90ee0f701346': City(name=UUID('abc911f5-bb28-44e6-97b9-90ee0f701346'), coordinates=[-72.34, -128.3], interests=[UUID('bb7baac3-dac6-4378-9d96-1b6c6e8ba917'), UUID('1a4501fb-f477-4119-893a-121c0cef8bf1'), UUID('373da6c1-44f9-4b92-a919-e4985c1f3c8b'), UUID('ba34724a-ab84-4f4a-a955-8072ca539fb8'), UUID('9cd65d5b-4cca-40db-8092-1d942562e270'), UUID('e439625c-bc4f-41f8-bfce-605e3ba249eb'), UUID('8c81a970-a9b6-44ca-9b9c-f0f58e3959e1')], country='aaa'), 'f4ddbd74-27ee-4281-ba48-45fe0bf3cc03': City(name=UUID('f4ddbd74-27ee-4281-ba48-45fe0bf3cc03'), coordinates=[16.9, -51.59], interests=[UUID('79d1650b-25e6-443c-bdbf-92ff41d751aa'), UUID('fc47fcb4-a2da-4c05-b26e-c3b37558b6d8'), UUID('fe19606c-75a7-4b10-8e65-8c8d0961d88f'


===== API 测试程序 =====

初始化数据...

准备测试数据...
已选择aaa国城市: 6191718d... 和 abc911f5...
已选择bbb国城市: 76e62c98... 和 d0448dc0...

=== 测试aaa国API ===

1. 测试 get_city_coordinates_aaa:
城市坐标: [-32.19, 25.51]
使用bbb国城市ID的结果: 错误：在aaa国找不到该城市

2. 测试 get_distance_between_cities_aaa:
城市间距离: 51.12
混合国家的结果: 错误：第二个城市不在aaa国

3. 测试 get_route_between_cities_aaa:
城市间路线: city_pair=(UUID('6191718d-6319-4ade-85e2-2dbc842842c8'), UUID('abc911f5-bb28-44e6-97b9-90ee0f701346')) steps=[UUID('1b678568-21ab-4eb5-8299-88ffa1e74c66'), UUID('571d2448-3414-43b1-9304-3d62e72c772b'), UUID('3f4681b8-de1c-47a3-815e-42c953dd1dfb')] distance=51.12

4. 测试 get_nearby_locations_aaa:
找到了 18 个附近城市
第一个附近城市: abc911f5-bb28-44e6-97b9-90ee0f701346

=== 测试bbb国API ===

1. 测试 get_city_coordinates_bbb:
城市坐标: [-44.8, -48.09]

2. 测试 get_distance_between_cities_bbb:
城市间距离: 76.94

3. 测试 get_route_between_cities_bbb:
城市间路线: 距离 76.94，步骤数 3

4. 测试 get_nearby_locations_bbb:
找到了 18 个附近城市

=== 测试跨国API ===

1. 测试 get_city_country:
aaa城市所属国家: aaa
bbb城市所属国家: bbb